<a href="https://colab.research.google.com/github/yoo9519-AIdev/Learning/blob/master/Seq2Seq_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 병렬 코퍼스와 전처리

In [1]:
import pandas as pd
import urllib3
import zipfile
import shutil
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [3]:
lines= pd.read_csv('fra.txt', names=['src', 'tar'], sep='\t')
len(lines)

177210

In [4]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 데애터가 너무 많으므로 6만개만 사용
lines.sample(10)

,src,tar
Let's see what happens.,Voyons voir ce qui arrive.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
That was embarrassing.,C'était embarrassant.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
We'll protect you.,Nous te protégerons.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
The donkey brayed.,L'âne brait.,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
I feel so lonely.,Je me sens si seule.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
Do you drink?,Buvez-vous ?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
He's active and fit.,Il est actif et en forme.,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
What is that?,Qu'est-ce ?,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
Watch closely.,Regardez de près !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
Don't eat without me.,Ne mange pas sans moi !,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


fra.txt 데이터는 왼쪽의 영어 문장과 오른쪽의 프랑스어 문장 사이에 탭으로 구분되는 구조가 하나의 샘플이다.  
앞으로의 코드에서 src는 입력 문장을 나타내며, tar는 번역하고자 하는 문장을 나타낸다.

In [5]:
# 프랑스 데이터에는 시작을 의미하는 '\t', 종료를 의미하는 '\n'심볼을 넣어주어야 한다.

lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
He has enough money.,Il dispose de suffisamment d'argent.,\t CC-BY 2.0 (France) Attribution: tatoeba.org...
He's after me.,Il est après mes fesses.,\t CC-BY 2.0 (France) Attribution: tatoeba.org...
I'm faithful to my wife.,Je suis fidèle envers ma femme.,\t CC-BY 2.0 (France) Attribution: tatoeba.org...
We count everything.,Nous comptons tout.,\t CC-BY 2.0 (France) Attribution: tatoeba.org...
Tom says he can do that.,Tom dit qu'il peut le faire.,\t CC-BY 2.0 (France) Attribution: tatoeba.org...
I slept very well.,J'ai très bien dormi.,\t CC-BY 2.0 (France) Attribution: tatoeba.org...
I'm volunteering.,Je fais du bénévolat.,\t CC-BY 2.0 (France) Attribution: tatoeba.org...
I'll attend.,Je serai présent.,\t CC-BY 2.0 (France) Attribution: tatoeba.org...
He is well off now.,"Il est riche, maintenant.",\t CC-BY 2.0 (France) Attribution: tatoeba.org...
Who wants jello?,Qui veut de la gelée ?,\t CC-BY 2.0 (France) Attribution: tatoeba.org...


글자 집합을 생성  
글자 집합을 생성하는 이유는 토큰 단위가 글자이기 때문

In [6]:
# 

In [7]:
# 글자 집합 구축

src_vocab=set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [8]:
# 글자 집합의 크기

src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

103
75


In [9]:
# 글자 집합에 글자 단위로 저장 된 것을 확인

src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['Z', '\\', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [10]:
# 이제 각 글자에 인덱스를 부여

src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '%': 4, '&': 5, "'": 6, '(': 7, ')': 8, ',': 9, '-': 10, '.': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, '\xa0': 76, '«': 77, '»': 78, 'À': 79, 'Ç': 80, 'É': 81, 'Ê': 82, 'Ô': 83, 'à': 84, 'â': 85, 'ç': 86, 'è': 87, 'é': 88, 'ê': 89, 'ë': 90, 'î': 91, 'ï': 92, 'ô': 93, 'ù': 94, 'û': 95, 'œ': 96, 'С': 97, '\u2009': 98, '\u200b': 99, '‘': 100, '’': 101, '\u202f': 102}
{'\t': 1, '\n': 2, ' ': 3, '#': 4, '&': 5, '(': 6, ')': 7, '-': 8, '.

In [11]:
# 인덱스가 부여된 글자 집합으로부터 갖고 있는 훈련 데이터에 대한 정수 인코딩 진행

encoder_input = []
for line in lines.src: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
      temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X)
print(encoder_input[:5])

[[45, 50, 1, 2], [42, 50, 61, 70, 69, 1, 2], [42, 50, 61, 70, 69, 11], [26, 64, 70, 67, 68, 102, 2], [26, 64, 70, 67, 54, 75, 102, 2]]


In [12]:
# 디코더에 입력값이 될 프랑스어 데이터에 대해서 정수 인코딩 진행

decoder_input = []
for line in lines.tar:
    temp_X = []
    for w in line:
      temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

[[1, 3, 23, 23, 8, 22, 45, 3, 12, 9, 10, 3, 6, 26, 66, 49, 62, 51, 53, 7, 3, 21, 68, 68, 66, 57, 50, 69, 68, 57, 63, 62, 20, 3, 68, 49, 68, 63, 53, 50, 49, 9, 63, 66, 55, 3, 4, 12, 18, 17, 17, 12, 17, 12, 3, 6, 23, 33, 7, 3, 5, 3, 4, 11, 11, 15, 18, 12, 15, 10, 3, 6, 43, 57, 68, 68, 73, 52, 53, 70, 7, 3, 2], [1, 3, 23, 23, 8, 22, 45, 3, 12, 9, 10, 3, 6, 26, 66, 49, 62, 51, 53, 7, 3, 21, 68, 68, 66, 57, 50, 69, 68, 57, 63, 62, 20, 3, 68, 49, 68, 63, 53, 50, 49, 9, 63, 66, 55, 3, 4, 15, 13, 18, 11, 12, 13, 3, 6, 23, 33, 7, 3, 5, 3, 4, 15, 10, 19, 18, 11, 19, 3, 6, 21, 57, 58, 57, 7, 3, 2], [1, 3, 23, 23, 8, 22, 45, 3, 12, 9, 10, 3, 6, 26, 66, 49, 62, 51, 53, 7, 3, 21, 68, 68, 66, 57, 50, 69, 68, 57, 63, 62, 20, 3, 68, 49, 68, 63, 53, 50, 49, 9, 63, 66, 55, 3, 4, 15, 13, 18, 11, 12, 13, 3, 6, 23, 33, 7, 3, 5, 3, 4, 14, 13, 12, 10, 14, 16, 12, 3, 6, 55, 57, 60, 60, 69, 72, 7, 3, 2], [1, 3, 23, 23, 8, 22, 45, 3, 12, 9, 10, 3, 6, 26, 66, 49, 62, 51, 53, 7, 3, 21, 68, 68, 66, 57, 50, 69, 68, 

디코더의 예측값과 비교할 실체값이 필요  
이 실체값에서는 '\t'이 필요 없다.  
(Dense, softmax 단어들 참고) 

In [13]:
decoder_target = []
for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
      if t>0:
        temp_X.append(tar_to_index[w])
      t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[3, 23, 23, 8, 22, 45, 3, 12, 9, 10, 3, 6, 26, 66, 49, 62, 51, 53, 7, 3, 21, 68, 68, 66, 57, 50, 69, 68, 57, 63, 62, 20, 3, 68, 49, 68, 63, 53, 50, 49, 9, 63, 66, 55, 3, 4, 12, 18, 17, 17, 12, 17, 12, 3, 6, 23, 33, 7, 3, 5, 3, 4, 11, 11, 15, 18, 12, 15, 10, 3, 6, 43, 57, 68, 68, 73, 52, 53, 70, 7, 3, 2], [3, 23, 23, 8, 22, 45, 3, 12, 9, 10, 3, 6, 26, 66, 49, 62, 51, 53, 7, 3, 21, 68, 68, 66, 57, 50, 69, 68, 57, 63, 62, 20, 3, 68, 49, 68, 63, 53, 50, 49, 9, 63, 66, 55, 3, 4, 15, 13, 18, 11, 12, 13, 3, 6, 23, 33, 7, 3, 5, 3, 4, 15, 10, 19, 18, 11, 19, 3, 6, 21, 57, 58, 57, 7, 3, 2], [3, 23, 23, 8, 22, 45, 3, 12, 9, 10, 3, 6, 26, 66, 49, 62, 51, 53, 7, 3, 21, 68, 68, 66, 57, 50, 69, 68, 57, 63, 62, 20, 3, 68, 49, 68, 63, 53, 50, 49, 9, 63, 66, 55, 3, 4, 15, 13, 18, 11, 12, 13, 3, 6, 23, 33, 7, 3, 5, 3, 4, 14, 13, 12, 10, 14, 16, 12, 3, 6, 55, 57, 60, 60, 69, 72, 7, 3, 2], [3, 23, 23, 8, 22, 45, 3, 12, 9, 10, 3, 6, 26, 66, 49, 62, 51, 53, 7, 3, 21, 68, 68, 66, 57, 50, 69, 68, 57, 63, 62, 

In [14]:
# 패딩 적용

max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

72
106


In [15]:
# 가장 긴 샘플의 길이에 맞춰서 패딩 적용

encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [16]:
# 모든 값에 대해서 One_Hot_Encoder 진행

encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

## Seq2Seq로 훈련

In [17]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [18]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.

encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

LSTM의 hidden layers는 256. 인코더의 내부 상태를 decoder로 넘겨주어야 하기 때문에  
return_state=True로 설정  
LSTM에서 state_h, state_c를 return받는데, 각각 은닉 상태와 셀 상태에 해당된다.  
이 두 가지 상태를 encoder_state에 저장한다.



In [19]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [21]:
# 입력으로 인코더와 디코더가 들어가고, 디코더의 실제값인 decoder_target도 필요하다. 배치는 60으로, 학습은 20으로(시간 관계 상) 조정
# 데이터가 과적합 된 상태로 진행

model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=20, validation_split=0.2)

Epoch 1/20
750/750 [==============================] - 19s 25ms/step - loss: 0.3691 - val_loss: 0.3569
Epoch 2/20
750/750 [==============================] - 18s 24ms/step - loss: 0.3406 - val_loss: 0.3402
Epoch 3/20
750/750 [==============================] - 18s 24ms/step - loss: 0.3264 - val_loss: 0.3314
Epoch 4/20
750/750 [==============================] - 18s 24ms/step - loss: 0.3162 - val_loss: 0.3247
Epoch 5/20
750/750 [==============================] - 18s 24ms/step - loss: 0.3082 - val_loss: 0.3186
Epoch 6/20
750/750 [==============================] - 18s 24ms/step - loss: 0.3018 - val_loss: 0.3160
Epoch 7/20
750/750 [==============================] - 18s 24ms/step - loss: 0.2961 - val_loss: 0.3115
Epoch 8/20
750/750 [==============================] - 18s 24ms/step - loss: 0.2909 - val_loss: 0.3090
Epoch 9/20
750/750 [==============================] - 18s 24ms/step - loss: 0.2862 - val_loss: 0.3066
Epoch 10/20
750/750 [==============================] - 18s 24ms/step - loss: 0.282

In [22]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 103)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 75)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 368640      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  339968      input_2[0][0]                    
                                                                 lstm[0][1]            

Seq2Seq 기계 번역기 동작시키기

전체적인 번역 동작 단계를 정리하면  
1. 번역하고자 하는 입력 문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻는다.  
2. 상태와 <SOS>에 해당하는 '\t'를 디코더로 보낸다.  
3. 디코더가 <EOS>에 해당하는 '\n'이 나올 때까지 다음 문자를 예측하는 행동을 반복한다.  

In [23]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [24]:
# 이전 시점의 상태들을 저장하는 텐서

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [25]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [27]:
# 단어로부터 인덱스를 얻는 것이 아니라, 인덱스로부터 단어를 얻을 수 있는 index_to_src와 index_to_tar를 만들었다.

def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [28]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Cours !
정답 문장:  CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906331 (sacredceltic) 
번역기가 번역한 문장:  CC-BY 2.0 (France) Attribution: tatoeba.org #2249602 (CK) & #3647320 (sacredceltic) 
-----------------------------------
입력 문장: J'ai menti.
정답 문장:  CC-BY 2.0 (France) Attribution: tatoeba.org #3935374 (gianich73) & #7581897 (Micsmithel) 
번역기가 번역한 문장:  CC-BY 2.0 (France) Attribution: tatoeba.org #2247402 (CK) & #4681769 (sacredceltic) 
-----------------------------------
입력 문장: Entre.
정답 문장:  CC-BY 2.0 (France) Attribution: tatoeba.org #348091 (Hertz) & #585174 (sacredceltic) 
번역기가 번역한 문장:  CC-BY 2.0 (France) Attribution: tatoeba.org #2249502 (CK) & #3647327 (sacredceltic) 
-----------------------------------
입력 문장: Magnez-vous !
정답 문장:  CC-BY 2.0 (France) Attribution: tatoeba.org #1329 (brauliobezerra) & #8691461 (sacredceltic) 
번역기가 번역한 문장:  CC-BY 2.0 (France) Attribution: tatoeba.org #2249502 (CK) & #3647340 (sacredceltic) 
